# LLM Initialization

In [ ]:
!pip install langchain huggingface_hub > /dev/null

In [ ]:
!pip install cohere loguru --quiet

In [ ]:
import os
huggingfacehub_api_token = ""

In [ ]:
from langchain import HuggingFaceHub

repo_id = "tiiuae/falcon-7b-instruct"
# repo_id = "bigscience/bloom"
falcon_llm = HuggingFaceHub(huggingfacehub_api_token=huggingfacehub_api_token,
                     repo_id=repo_id,
                     model_kwargs={"temperature":0.6, "max_new_tokens":500})

In [ ]:
# from langchain import PromptTemplate, LLMChain

# template = """
# You are an ai assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

# {question}
# """
# prompt = PromptTemplate(template=template, input_variables=["question"])
# llm_chain = LLMChain(prompt=prompt, llm=falcon_llm)

# question = "what is machine learning?"

# print(llm_chain.run(question))

In [ ]:
import os
cohere_api_token = ""

In [ ]:
from langchain.llms import Cohere
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

cohere_llm = Cohere(model='command',cohere_api_key=cohere_api_token)
# llm_chain = LLMChain(prompt=prompt, llm=cohere_llm)

# question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

# llm_chain.run(question)

# Callbacks

In [ ]:
!pip install langchain_experimental==0.0.1 --q

In [ ]:
from langchain.utilities import SQLDatabase
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [ ]:
## Initialize SQLDatabaseChain using Chinook database and LangChain

In [ ]:
db = SQLDatabase.from_uri("sqlite:///../../../../content/Chinook.db")
llm = cohere_llm

In [ ]:
db.table_info

'\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/\n\n\nCREATE TABLE "Artist" (\n\t"ArtistId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("ArtistId")\n)\n\n/*\n3 rows from Artist table:\nArtistId\tName\n1\tAC/DC\n2\tAccept\n3\tAerosmith\n*/\n\n\nCREATE TABLE "Customer" (\n\t"CustomerId" INTEGER NOT NULL, \n\t"FirstName" NVARCHAR(40) NOT NULL, \n\t"LastName" NVARCHAR(20) NOT NULL, \n\t"Company" NVARCHAR(80), \n\t"Address" NVARCHAR(70), \n\t"City" NVARCHAR(40), \n\t"State" NVARCHAR(40), \n\t"Country" NVARCHAR(40), \n\t"PostalCode" NVARCHAR(10), \n\t"Phone" NVARCHAR(24), \n\t"Fax" NVARCHAR(24), \n\t"Email" NVARCHAR(60) NOT NULL, \n\t"SupportRepId" INTEG

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
db_chain.run("How many employees are there?")



> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:SELECT Count(1) FROM Employee;
SQLResult: [(8,)]
Answer:(8)
> Finished chain.


'(8)'

In [ ]:
db_chain.run("how many rows are there in revenue table")



> Entering new SQLDatabaseChain chain...
how many rows are there in revenue table
SQLQuery:SELECT Count(*) FROM revenue

OperationalError: ignored

In [ ]:
try:
  db_chain.run("how many rows are there in revenue table")
except Exception as e:
    print("Error occurred")



> Entering new SQLDatabaseChain chain...
how many rows are there in revenue table
SQLQuery:SELECT count(*) FROM "revenue"Error occurred


In [ ]:
## Using CustomHandler to create callbacks while SQLChain execution

In [ ]:
from langchain.callbacks.base import BaseCallbackHandler
from typing import Any, Dict, List, Optional, Union

class MyCustomHandler(BaseCallbackHandler):

    def on_chain_start(self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any) -> Any:
        print("Starting SQL query!")

    def on_chain_end(self, output: Dict[str, Any], **kwargs: Any) -> Any:
        print("SQL query complete!")

In [ ]:
my_handler = MyCustomHandler()
db_chain.run("How many employees are there?",callbacks=[my_handler])

Starting SQL query!


> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:Starting SQL query!
SQL query complete!
SELECT Count(EmployeeId) FROM Employee
SQLResult: [(8,)]
Answer:Starting SQL query!
SQL query complete!
8SQL query complete!

> Finished chain.


'8'

In [ ]:
## Creating Custom SQLErrorCallback to print error during SQLChain execution

In [ ]:
from langchain.callbacks.base import BaseCallbackHandler

class SQLErrorCallback(BaseCallbackHandler):

    def on_chain_error(self, error: Exception, **kwargs: Any) -> Any:
        print(f"SQL Error: {error} is the error")

In [ ]:
error_handler = SQLErrorCallback()
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, callbacks=[error_handler])

In [ ]:
try:
    db_chain.run("how many columns are there in revenue table")
except Exception as e:
    print("Error occurred")



> Entering new SQLDatabaseChain chain...
how many columns are there in revenue table
SQLQuery:SELECT Count(*) FROM revenueSQL Error: (sqlite3.OperationalError) no such table: revenue
[SQL: SELECT Count(*) FROM revenue]
(Background on this error at: https://sqlalche.me/e/20/e3q8) is the error
Error occurred


In [ ]:
## Using loguru and LangChain's FileCallbackHandler to log errors to file

In [ ]:
from loguru import logger
from langchain.callbacks import FileCallbackHandler

error_logfile = "sql_errors.log"
logger.add(error_logfile, colorize=True, enqueue=True)

error_handler = FileCallbackHandler(error_logfile)

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, callbacks=[error_handler])

try:
    db_chain.run("how many columns are there in revenue table")
except Exception as e:
    logger.error(e)





> Entering new SQLDatabaseChain chain...
how many columns are there in revenue table
SQLQuery:

2023-07-29 07:26:23.938 | ERROR    | __main__:<cell line: 11>:14 - (sqlite3.OperationalError) no such table: revenue
[SQL: SELECT count(*) FROM revenue]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


SELECT count(*) FROM revenue

# Tracing/Evaluation

In [ ]:
# wandb.ai tracing
!pip install "wandb>=0.15.4" -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00


In [ ]:
import langchain

In [ ]:
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
os.environ["WANDB_PROJECT"] = "langchain-testing"

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# This is an LLMChain to write a synopsis given a title of a play.
template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=falcon_llm, prompt=prompt_template)

# This is an LLMChain to write a review of a play given a synopsis.
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=cohere_llm, prompt=prompt_template)

# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [ ]:
# some sample play titles
titles = [
  "Tragedy at sunset on the beach",
  "Death of a Salesman"
]

for title in titles:
  try:
    # call your Agent as normal
    answer = overall_chain.run(title)
    print(answer)
  except Exception as e:
    # any errors will be also logged to Weights & Biases
    print(e)
    pass

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Streaming LangChain activity to W&B at https://wandb.ai/brewery-controller/langchain-testing/runs/8q8gkvw8
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.




> Entering new SimpleSequentialChain chain...


The sun is setting on the beach, and two old friends, George and Martha, have come together to share stories of their lives and to reconcile their differences. As they look back on their past and reflect on the sunset's beauty, they realize the importance of the friendship that has lasted for decades. The play explores the themes of aging, mortality, and the value of friendship amidst the fading of the sun.

The setting sun on the beach is a fitting backdrop for the two old friends, George and Martha, as they reflect on their lives and the friendship that has lasted for decades. Their stories are a testament to the power of friendship, even in the face of aging and mortality. The play is a reminder of the beauty and importance of sunset moments, when we can appreciate the fleeting nature of life and the value of the relationships we hold dear.

> Finished chain.

The setting sun on the beach is a fitting backdrop for the two old friends